## Generacion de resumenes mediante bart-large-cnn

- El codigo ha sido extraido de: https://huggingface.co/facebook/bart-large-cnn
- El codigo ha sido extraido de: https://huggingface.co/google/bigbird-pegasus-large-arxiv

### Primera prueba mediante el pipeline de transformers

In [22]:
# Ejemplo extraido del notebook DOCLING_IBM
txt = """ We introduce Florence-2 , a novel vision foundation model with a unified, prompt-based representation for a variety of computer vision and 
vision-language tasks. While existing large vision models excel in transfer learning, they struggle to perform a diversity of tasks with simple instructions,
a capability that implies handling the complexity of various spatial hierarchy and semantic granularity. Florence-2 was designed to take text-prompt as task
instructions and generate desirable results in text forms, whether it be captioning, object detection, grounding or segmentation. This multi-task learning
setup demands largescale, high-quality annotated data. To this end, we codeveloped FLD-5B that consists of 5.4 billion comprehensive visual annotations
on 126 million images, using an iterative strategy of automated image annotation and model refinement. We adopted a sequence-to-sequence structure to 
train Florence-2 to perform versatile and comprehensive vision tasks. Extensive evaluations on numerous tasks demonstrated Florence-2 to be a strong vision 
foundation model contender with unprecedented zero-shot and fine-tuning capabilities.
"""

In [23]:
# Leo el archivo generado en el otro notebook
with open(r"..\Segmentacion_PDF\texto_para_resumen.txt", "r", encoding="utf-8") as archivo:
    texto_grande = archivo.read()

### bart-large-cnn - Numero de tokens 1024 tokens

In [25]:
from transformers import pipeline
from rouge_score import rouge_scorer
from transformers import AutoTokenizer

# utilizo la API de hugginface para cargar el modelo
summarizer_bart = pipeline("summarization", model="facebook/bart-large-cnn", device=0)
tokenizer_bart = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

Device set to use cuda:0


In [26]:
tokens = tokenizer_bart.encode(texto_grande)
print(f"Número de tokens: {len(tokens)}")

Número de tokens: 22281


In [27]:
resumen = summarizer_bart(txt,max_length=200, min_length=30, do_sample=False)[0]['summary_text']
print(resumen)

Florence-2 is a novel vision foundation model with a unified, prompt-based representation for a variety of computer vision and vision-language tasks. Florence-2 was designed to take text-prompt as task instructions and generate desirable results in text forms. This multi-task learningsetup demands largescale, high-quality annotated data.


In [28]:
# Evaluacion del resumen con mediante ROUGE

# Generamos el objeto con las 3 mmetricas:
    # Rouge 1: Evalua la coincidencia de unigramas (palabras individuales), midiendo cuántas palabras del resumen aparecen en el texto original.
    # Rouge 2: Evalua la coincidencia de bigramas (pares de palabras consecutivas), midiendo cuantos pares de palabras del resumen se encuentran en el texto original.
    # Rouge L: Evalúa la coincidencia de la subsecuencia común más larga, que es la secuencia más larga de palabras que aparece en ambos textos en el mismo orden,
    #            aunque no necesariamente de forma contigua.
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

scores = scorer.score(txt, resumen)


print(f'Rouge 1\n\tPrecision : {round(scores["rouge1"][0],2)}\n\tRecall : {round(scores["rouge1"][1],2)}\n\tF1 Score : {round(scores["rouge1"][2],2)}')
print(f'Rouge 2\n\tPrecision : {round(scores["rouge2"][0],2)}\n\tRecall : {round(scores["rouge2"][1],2)}\n\tF1 Score : {round(scores["rouge2"][2],2)}')
print(f'Rouge L\n\tPrecision : {round(scores["rougeL"][0],2)}\n\tRecall : {round(scores["rougeL"][1],2)}\n\tF1 Score : {round(scores["rougeL"][2],2)}')


Rouge 1
	Precision : 0.96
	Recall : 0.29
	F1 Score : 0.45
Rouge 2
	Precision : 0.88
	Recall : 0.27
	F1 Score : 0.41
Rouge L
	Precision : 0.96
	Recall : 0.29
	F1 Score : 0.45


- Precision: cuántas palabras del resumen coinciden con el texto original
- Recall: cuántas palabras del texto original están cubiertas por el resumen
- F1 Score: balance entre la precisión y el recall

Analisis metricas:

 - Rouge 1: Se observa que el modelo utiliza el 96% de las palabras en el resumen existen en el texto original y 29% de las palabras del texto original se encuentran en el resumen.
 - Rouge 2: Se observa que el modelo utilzia el 88% de los bigramas del resumen existen en el texto original y 27% de los bigramas del texto original se encuentran en el resumen.
 - Rouge L: Se observa que el modelo utilzia el 96% de las secuencias mas largas del resumen existen en el texto original y 29% de las secuencias mas largas del texto original se encuentran en el resumen.

En este ejemplo, se puede concluir que este modelo puede ser utilizado para el uso de generacion de resumenes. A continuacion, se realizara una prueba con un texto de mayor longitud y complejidad.

In [29]:
tokens = tokenizer_bart.encode(texto_grande[5000:10000])
print(f"Número de tokens: {len(tokens)}")

Número de tokens: 1005


In [31]:
# Incremento el maximo del output
summarizer_bart(texto_grande[5000:10000],max_length=600, min_length=30, do_sample=False)[0]['summary_text']

'Florence-2 is capable of performing a variety of tasks, such as object detection, captioning, and grounding, all within a single model governed by a uniform set of parameters. The pre-trained Florence-2 backbone enhances performance on downstream tasks, e.g. COCO object detection and instance segmentation, and ADE20K semantic segmentation.'

Si introduzco mas de los tokens que el modelo permite, da error.

### BigBirdPegasus - Numero de tokens 4096 tokens

In [53]:
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-arxiv")

model_Pegasus_OF = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv",attention_type="original_full",block_size=16, num_random_blocks=2)
model_Pegasus_SparseAttention = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv",block_size=16, num_random_blocks=2)

In [56]:
inputs = tokenizer(txt, return_tensors='pt')
prediction_OF = model_Pegasus_OF.generate(**inputs,)
prediction_OF = tokenizer.batch_decode(prediction_OF)
prediction_OF

['<s> we introduce a novel foundation model with a unified prompt - based representation for a variety of computer vision and vision tasks .</s>']

In [ ]:
scores = scorer.score(txt, str(prediction_OF))

print(f'Rouge 1\n\tPrecision : {round(scores["rouge1"][0],2)}\n\tRecall : {round(scores["rouge1"][1],2)}\n\tF1 Score : {round(scores["rouge1"][2],2)}')
print(f'Rouge 2\n\tPrecision : {round(scores["rouge2"][0],2)}\n\tRecall : {round(scores["rouge2"][1],2)}\n\tF1 Score : {round(scores["rouge2"][2],2)}')
print(f'Rouge L\n\tPrecision : {round(scores["rougeL"][0],2)}\n\tRecall : {round(scores["rougeL"][1],2)}\n\tF1 Score : {round(scores["rougeL"][2],2)}')

Rouge 1
	Precision : 0.91
	Recall : 0.12
	F1 Score : 0.22
Rouge 2
	Precision : 0.82
	Recall : 0.11
	F1 Score : 0.19
Rouge L
	Precision : 0.91
	Recall : 0.12
	F1 Score : 0.22


In [ ]:
inputs = tokenizer(txt, return_tensors='pt')
prediction_Sparseattention = model_Pegasus_SparseAttention.generate(**inputs)
prediction_Sparseattention = tokenizer.batch_decode(prediction_Sparseattention)
prediction_Sparseattention

['<s> in this paper , we introduce largeshot-@xmath0 , a novel computer vision model with a unified prompt model capabilities .<n> largeshot-@xmath0 is a sequence-@xmath0 model , where @xmath1 is the number of training frames , @xmath2 is the number of test frames , and @xmath3 is the number of test images .<n> largeshot-@xmath0 is a sequence-@xmath0 learning model , where @xmath1 is the number of training frames , @xmath2 is the number of test frames , and @xmath3 is the number of test images .<n> largeshot-@xmath0 is a sequence-@xmath0 learning model , where @xmath1 is the number of training frames , @xmath2 is the number of test frames , and @xmath3 is the number of test images .<n> largeshot-@xmath0 is a sequence-@xmath0 learning model , where @xmath1 is the number of training frames , @xmath2 is the number of test frames , and @xmath3 is the number']

In [58]:
scores = scorer.score(txt, str(prediction_Sparseattention))

print(f'Rouge 1\n\tPrecision : {round(scores["rouge1"][0],2)}\n\tRecall : {round(scores["rouge1"][1],2)}\n\tF1 Score : {round(scores["rouge1"][2],2)}')
print(f'Rouge 2\n\tPrecision : {round(scores["rouge2"][0],2)}\n\tRecall : {round(scores["rouge2"][1],2)}\n\tF1 Score : {round(scores["rouge2"][2],2)}')
print(f'Rouge L\n\tPrecision : {round(scores["rougeL"][0],2)}\n\tRecall : {round(scores["rougeL"][1],2)}\n\tF1 Score : {round(scores["rougeL"][2],2)}')

Rouge 1
	Precision : 0.27
	Recall : 0.22
	F1 Score : 0.24
Rouge 2
	Precision : 0.06
	Recall : 0.05
	F1 Score : 0.06
Rouge L
	Precision : 0.19
	Recall : 0.16
	F1 Score : 0.17


Se observa que el rendimiento de BigBird con el tipo de atención original_full se asemeja al de BART, aunque con resultados ligeramente inferiores. Por otro lado, al utilizar el tipo de atención sparse, se obtiene la capacidad de manejar una mayor cantidad de tokens, lo que podría ser útil para reconstruir los documentos mediante plantillas. Sin embargo, esto conlleva un aumento significativo en la complejidad programática y, por ende, un mayor consumo computacional. Este es uno de los principales desafíos del proyecto, por lo que, de momento, no es una orientación viable. En consecuencia, se concluye que, a priori, el modelo BART es más adecuado para las necesidades de Lervis.